# **Unsupervised Learning Trading Strategy**

### **1. Download/Load SP500 stocks prices data** 

In [6]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings

warnings.filterwarnings("ignore")



In [20]:
# 1. Get S&P 500 tickers
sp500 = pd.read_html(
    "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",
    storage_options={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36"
    }
)[0]

sp500["Symbol"] = sp500["Symbol"].str.replace(".", "-", regex=False)
symbols_list = sp500["Symbol"].unique().tolist()


# 2. Define date range
end_date = "2025-12-15"
start_date = pd.to_datetime(end_date) - pd.DateOffset(days=365 * 8)


# 3. Download price data
df = yf.download(
    tickers=symbols_list,
    start=start_date,
    end=end_date,
    auto_adjust=False
)


# 4. Reshape to match the course format
df = (
    df
    .stack()                                # wide → long
    .rename_axis(["date", "ticker"])        # name index levels
)

df.columns = df.columns.str.lower()         # lowercase column names

df.head()


[*********************100%***********************]  503 of 503 completed


Price              adj close      close       high        low       open  \
date       ticker                                                          
2017-12-18 A       63.697105  67.660004  68.510002  67.480003  67.889999   
           AAPL    41.316132  44.105000  44.299999  43.715000  43.720001   
           ABBV    69.745361  98.190002  99.099998  97.550003  97.669998   
           ABT     49.003803  56.400002  56.520000  55.419998  55.520000   
           ACGL    28.961201  30.456667  30.853333  30.416668  30.726667   

Price                   volume  
date       ticker               
2017-12-18 A         2306300.0  
           AAPL    117684400.0  
           ABBV      4768300.0  
           ABT       6053900.0  
           ACGL      1212300.0

# **2. Calculate features and technical indicators for each stock**

- Garman-Klass Volatility  
- RSI (relative strength index)
- Bollinger Bands (lower upper and middle bands)
- ATR (Average true range)
- MACD  
- Dollar Volume  

#### Garman-Klass Volatility

$$
\sigma_{GK}^2 =
\frac{(\ln(H) - \ln(L))^2}{2}
- (2\ln(2) - 1)(\ln(C) - \ln(O))^2
$$

Where:
- $H$ = High  
- $L$ = Low  
- $C$ = Adjusted Close  
- $O$ = Open  


In [30]:
# df["garman_klass_vol"] = ((np.log(df["high"]) - np.log(df["low"]))**2 / 2 - (2 * np.log(2) - 1) * (np.log(df["adj close"]) - np.log(df["open"]))**2)

# df["rsi"] = (df.groupby(level=1)["adj close"].transform(lambda x: pandas_ta.rsi(close=x, length=20)))

df["bb_low"] = df.groupby(level=1)["adj close"].transform(
    lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:, 0]
)

df["bb_mid"] = df.groupby(level=1)["adj close"].transform(
    lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:, 1]
)

df["bb_high"] = df.groupby(level=1)["adj close"].transform(
    lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:, 2]
)
df.head()



Price              adj close      close       high        low       open  \
date       ticker                                                          
2017-12-18 A       63.697105  67.660004  68.510002  67.480003  67.889999   
           AAPL    41.316132  44.105000  44.299999  43.715000  43.720001   
           ABBV    69.745361  98.190002  99.099998  97.550003  97.669998   
           ABT     49.003803  56.400002  56.520000  55.419998  55.520000   
           ACGL    28.961201  30.456667  30.853333  30.416668  30.726667   

Price                   volume  garman_klass_vol  rsi  bb_low  bb_mid  bb_high  
date       ticker                                                               
2017-12-18 A         2306300.0         -0.001455  NaN     NaN     NaN      NaN  
           AAPL    117684400.0         -0.001147  NaN     NaN     NaN      NaN  
           ABBV      4768300.0         -0.043680  NaN     NaN     NaN      NaN  
           ABT       6053900.0         -0.005828  NaN     NaN     NaN      NaN  
           ACGL      1212300.0         -0.001251  NaN     NaN     NaN      NaN

### **3. Calculate monthly returns**